In [3]:
#we will import from DataandTokenizer, we have everything setup as if it was a non jupyter notebook
#explanations can be found of Basic Manipulation with explanation

%run DataandTokenizer.ipynb

#Now that we have it run we can go ahead and start preparing our transformer 


## Usage of class Embed
Embedding has Embedding.tokenize(data) ->(tokenizedspanish(1 per word),tokenizedpt)as dtype= Tensor


Embedding has Embedding.detokenize(data) ->(detokenizedspanish,detokenizedpt) as dtype= Tensor

Data has to have dimensions (2(spanish,portuguese),datadimension) --> [(esdata),(ptdata)]


Be careful with detokenized as it should be decoded again!
For now they are only crated for the specific case of spanish and pt.

It will include de START AND END ALSO in both 

## Usage of class Data
Data() -> Data.pt ->(Data pt) dimension(words)


       -> Data.es ->(Data es) dimension(words)

In [2]:
#testin

Datos=Data()
Embed=Embedding()


In [3]:
#TO UNDERSTAND DIMENSIONALITY
print(Datos.es[12])
test=[]
testes=[]
testpt=[]
testes.append(Datos.es[12])
testpt.append(Datos.pt[12])

testes.append(Datos.es[13])
testpt.append(Datos.pt[13])
test.append(testes)
test.append(testpt)
print(Embed.tokenize(test))
print(Embed.detokenize(Embed.tokenize(test)))



`` no dicen , `` '' quiero mejor agua caliente en las duchas . ''
[<tf.Tensor: shape=(2, 23), dtype=int64, numpy=
array([[   2,   38,   38,  197,  776,   14,   38,   38,    9,    9,  324,
         333,  442, 2805,  191,  200,   42, 1070, 2101,   16,    9,    9,
           3],
       [   2,  282,   16,   10,  281,   11,    3,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0]], dtype=int64)>, <tf.Tensor: shape=(2, 24), dtype=int64, numpy=
array([[   2,   39,   39,  162,  117,  658,   28,   39,   39,    9,    9,
         264,  373, 1950,  133,   42,  523,  201,  557, 1674,   16,    9,
           9,    3],
       [   2,  282,   16,   10,  209,   11,    3,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0]], dtype=int64)>]
[<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b"[START] ` ` no dicen , ` ` ' ' quiero mejor agua caliente en las duchas . ' ' [END]",
  

Now following the structure of the transformer we need a positional codificator. We will use the one described in https://arxiv.org/abs/1706.03762

In [19]:
# we can precompute all the positional arguments in a single numpy array 

def angle(pos,i,dimension):
    return pos / np.power(10000, (2 * (i//2)) / np.float32(dimension))
   
def positional(pos,dimension):
    posmatrix=np.arange(pos)[:,np.newaxis]
    dimmatrix=np.arange(dimension)[np.newaxis,:]
    angles=angle(posmatrix,dimmatrix,dimension)
    i=0
    for substack in angles:
        i=0
        for value in substack:
            if i%2==0:
                value=np.sin(value)#~we apply what is desired
            else:
                value=np.cos(value)
            i+=1
    
    #this will return a [ [](dimensions) ,... []] for every position we have dimension values think it as an embedding dimension
    return angles
#And we have or positional embedding fully defined

We will also need padding mask and another mask to not let it read ahead.
The padding masked just makes 0 if its non padding, meaning its non zero value, and 1 if its 0.


In [16]:
def paddingmask(data):
    #data will have dims (batchsize, sequence)
    #for attention layers we will include two extra dimensions(batchsize,1,1,sequence)
    data = tf.cast(tf.math.equal(data, 0), tf.float32)
    #we do math equal in case of true cast will make it a 1.0 otherwise it will be false and cast into 0.0
    
    return data[:,tf.newaxis,tf.newaxis,:]

def lookahead(dimension): #Dimensions has shape (dim,dim)
    #what we want is something that has first 0.0 and 1.0 for all values in the first row and then add another 0.0 for each row until 
    #completed
    #we will give dimension of the maximum sequence length ie(1,5) a sequence of five words 
    #in general we will have a (sequence,sequence) output dimension
    res=np.ones(dimension)
    for index,value in enumerate(res):
        res[index:,index]=0.0
        #we set from (x,x) to below as 0
        
    return tf.convert_to_tensor(res)

    


We put 0, where one would find reasonable to put 1. This is because we multiply by -inf to create a similar result when applying softmax. That is a 0 is going to have much more impact

Now we are ready to tackle the attention part of the paper.


# Dot product attention
The structure and formulas can be found on the paper. We need some set of Queries, Keys and Values that encapsulate the information between relationship of words and the value of a set word in a sentence.


A relly important step is that we sum the mask to the result that we are achieving multiplied to a value close to -inf. 

In [82]:
def dotproductattention(q,k,v,mask):
    
    
    #First we need to matmul q,k transposing k
    resmatmul=tf.matmul(q,k,transpose_b=True)
    
    #then we need the dimension of k d_k 
    kdim=float(tf.shape(k)[-1])
    resmatmul=resmatmul/(tf.math.sqrt(kdim))
    
    #It may be useful to include the situation where mask is None
    if mask is not None:
        resmatmul += mask*-1e9
    
    ressoftmax=tf.nn.softmax(resmatmul,axis=-1) # we need to do softmax on the last dimension
    return tf.matmul(ressoftmax,v), ressoftmax
    
    

In [83]:
temp_k = tf.constant([[10, 0, 0],
                      [0, 10, 0],
                      [0, 0, 10],
                      [0, 0, 10]], dtype=tf.float32)  # (4, 3)

temp_v = tf.constant([[1, 0],
                      [10, 0],
                      [100, 5],
                      [1000, 6]], dtype=tf.float32)  # (4, 2)

# This `query` aligns with the second `key`,
# so the second `value` is returned.
temp_q = tf.constant([[10, 0, 0]], dtype=tf.float32)  # (1, 3)

In [84]:
dotproductattention(temp_q,temp_k,temp_v,None)

(<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[1.000000e+00, 9.276602e-25]], dtype=float32)>,
 <tf.Tensor: shape=(1, 4), dtype=float32, numpy=
 array([[1.0000000e+00, 8.4332744e-26, 8.4332744e-26, 8.4332744e-26]],
       dtype=float32)>)

All that remains now is to create the multi head attention. The powerful idea of multi-head is the usage of parallel computation so what the papers does is project in a number of dotproductattention layers, and then concatanate the result


Also each queries,keys and values will need their respective weights as described in the paper. That is a keras Dense model(lineal). We will call the dimension of these weights the modeldim. 



We need to create a class that is a Layer of keras that is


In [89]:
class MultiAttention(tf.keras.layers.Layer):
    def __init__(self,modeldim,layersnum): 
        # we need to inherit all of the keras information of th super
        super().__init__()
        #now we can start working
        
        #WEIGHTS FOR Q,K,V
        self.wq=tf.keras.layers.Dense(modeldim)
        self.wk=tf.keras.layers.Dense(modeldim)
        self.wv=tf.keras.layers.Dense(modeldim)
                          
            
        #WEIGHTS FO AFTER CONCATENTION
        self.dense=tf.keras.layers.Dense(modeldim)
        
        #extra information
        
        self.modeldim=modeldim
        self.layersnum=layersnum
        assert d_model % self.num_heads == 0
        self.depth=d_model//self.num_heads
        
        
    #WE NEED TO SPLIT INTO modeldim /layresnum
    
    #All this part is heavily optimized thanks to the tensorflow tutorial for transformers
    #The main idea is to split the last dimension into (number of layers, model dimension)
    def split_heads(self, x, batch_size):
        #Split the last dimension into (num_heads, depth).
        #Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        #
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights

In [90]:
Mul=MultiAttention(10,10)

In [78]:
Mul.split